In [2]:
import datetime
import os
import urllib
from prettytable import PrettyTable
import pandas as pd
import glob


print("Setup ok")

Setup ok


In [17]:
import os
import urllib.request
import datetime

def download_data(province_id, year_start=1981, year_end=2024):
    # перевіряємо чи існує папка 'data'
    data_folder = 'data'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    else:
        # перевіряємо чи існує завантажений файл
        existing_files = [file for file in os.listdir(data_folder) if file.startswith(f'vhi_id__{province_id}__')]
        if existing_files:
            print(f"Файл '{existing_files[0]}' вже перебуває в папці 'data', додаткове завантаження не потрібне.")
            return

    # завантаження даних
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year_start}&year2={year_end}&type=Mean"
    response = urllib.request.urlopen(url)

    # запис даних у файл
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    filename = f'vhi_id__{province_id}__{current_datetime}.csv'
    file_path = os.path.join(data_folder, filename)

    with open(file_path, 'wb') as file:
        file.write(response.read())

    print(f"VHI завантажений та збережний у файлі '{file_path}'")

In [18]:
for i in range(1,28):
   download_data(i)

Файл 'vhi_id__1__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__2__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__3__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__4__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__5__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__6__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__7__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__8__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__9__2025-06-05_10-58.csv' вже перебуває в папці 'data', додаткове завантаження не потрібне.
Файл 'vhi_id__10__2025-06-05_10-58.csv' вже перебуває в

In [7]:
reg_id_name = {
    1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
    6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
    11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
    16: 'Рівненська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
    21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
}

In [8]:
def create_data_frame(folder_path):

    csv_files = glob.glob(folder_path + "/*.csv")

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    frames = []

    for file in csv_files:
        region_id = int(file.split('__')[1])
        df = pd.read_csv(file, header = 1, names = headers)
        df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
        df=df.drop(df.index[-1])
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df.insert(0, 'region_id', region_id, True)
        df['Week'] = df['Week'].astype(int)
        
        frames.append(df)   
    result = pd.concat(frames).drop_duplicates().reset_index(drop=True)

    result = result.loc[(result.region_id != 12) & (result.region_id != 20)]
    result = result.replace({'region_id':{1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21,
                                          11:9, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17, 
                                          22:18, 23:6, 24:1, 25:2, 26:6, 27:5}})
    return result

In [9]:
resultdf = create_data_frame('data')

print(resultdf)

       region_id  Year  Week    SMN     SMT    VCI    TCI    VHI
0             21  1982     1  0.059  258.24  51.11  48.78  49.95
1             21  1982     2  0.063  261.53  55.89  38.20  47.04
2             21  1982     3  0.063  263.45  57.30  32.69  44.99
3             21  1982     4  0.061  265.10  53.96  28.62  41.29
4             21  1982     5  0.058  266.42  46.87  28.57  37.72
...          ...   ...   ...    ...     ...    ...    ...    ...
59017         20  2024    48  0.135  278.17  55.23  11.86  33.55
59018         20  2024    49  0.133  277.08  57.71  10.86  34.29
59019         20  2024    50  0.130  276.49  59.45   8.68  34.07
59020         20  2024    51  0.128  276.45  62.53   5.55  34.04
59021         20  2024    52  0.129  276.48  66.13   3.71  34.92

[54650 rows x 8 columns]


In [20]:
def region_year_analysis(region_id, years =("1982","2024"), ):

    df2 = resultdf[(resultdf["Year"].between(years[0],years[1])) & (resultdf['region_id'] == region_id)]
    print(df2)
    region_name = reg_id_name[region_id]
    vhi_max_reg = df2["VHI"].max()
    vhi_min_reg = df2["VHI"].min()
    print(f"[+] {region_name}: min {vhi_min_reg} max {vhi_max_reg}")

In [21]:
years = ("2000","2000")
for i in range(1,26):
    region_year_analysis(i)

       region_id  Year  Week    SMN     SMT    VCI    TCI    VHI
32790          1  1982     1  0.068  263.59  63.47  28.34  45.90
32791          1  1982     2  0.074  265.78  67.62  23.05  45.34
32792          1  1982     3  0.076  267.19  69.37  20.40  44.88
32793          1  1982     4  0.075  268.57  65.26  17.93  41.60
32794          1  1982     5  0.072  269.24  58.58  20.00  39.29
...          ...   ...   ...    ...     ...    ...    ...    ...
34971          1  2024    48  0.100  270.83  57.76  26.51  42.13
34972          1  2024    49  0.086  268.23  55.83  29.40  42.61
34973          1  2024    50  0.076  266.10  53.85  28.50  41.18
34974          1  2024    51  0.068  264.86  53.12  25.29  39.20
34975          1  2024    52  0.070  265.65  59.34  19.50  39.42

[2186 rows x 8 columns]
[+] Вінницька: min 11.25 max 82.64
       region_id  Year  Week    SMN     SMT    VCI    TCI    VHI
34976          2  1982     1  0.061  255.81  28.81  78.96  53.88
34977          2  1982     2  

In [14]:
def drought_years(df, percentage, type_of_drought, start_year=None, end_year=None):
    # якщо рік не був заданий - ставимо початковий або останній рік датафрейму
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]

    # словник кількості областей з VHI менше порогового значення для кожного року
    drought_count = {}

    # групую дані за роками
    grouped_data = df.groupby('Year')

    # перебираю кожний рік
    for year, group_df in grouped_data:
    
        # знаходжу к-сть областей у цьому році
        total_areas = len(group_df['region_id'].unique())

        if type_of_drought == "extreme":
            # к-сть областей з VHI < 15
            drought_areas = group_df[group_df['VHI'] < 15]['region_id'].nunique()
        elif type_of_drought == "moderate":
            # к-сть областей з VHI > 15 але < 35
            drought_areas = group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['region_id'].nunique()
        
        # якщо к-сть посушливих областей > заданий відсоток - то зберігаю рік
        if drought_areas / total_areas > percentage / 100:
            drought_count[year] = (drought_areas, round((drought_areas / total_areas) * 100, 2))

    # створюю таблицю за допомогою PrettyTable
    table = PrettyTable()
    if type_of_drought == "extreme":
        table.title = f"Роки з екстремальною посухою (де більше {percentage}% областей мають VHI < 15)"
    elif type_of_drought == "moderate":
        table.title = f"Роки з помірною посухою (де більше {percentage}% областей мають VHI від 15 до 35)"

    table.field_names = ["Year", "К-сть областей з посухою", "Відсоток"]

    for year, (drought_areas, percent) in drought_count.items():
        table.add_row([year, drought_areas, f"{percent}%"])

    return table

# приклад для екстримальних
drought_years_extreme = drought_years(resultdf, 10, "extreme", start_year='2000', end_year='2024')
print(drought_years_extreme)

# приклад для помірних
drought_years_moderate = drought_years(resultdf, 10, "moderate", start_year='2000', end_year='2024')
print(drought_years_moderate)


+----------------------------------------------------------------------+
| Роки з екстремальною посухою (де більше 10% областей мають VHI < 15) |
+--------+-------------------------------------------+-----------------+
|  Year  |          К-сть областей з посухою         |     Відсоток    |
+--------+-------------------------------------------+-----------------+
|  2000  |                     4                     |      16.67%     |
|  2007  |                     5                     |      20.83%     |
+--------+-------------------------------------------+-----------------+
+-------------------------------------------------------------------------+
| Роки з помірною посухою (де більше 10% областей мають VHI від 15 до 35) |
+---------+---------------------------------------------+-----------------+
|   Year  |           К-сть областей з посухою          |     Відсоток    |
+---------+---------------------------------------------+-----------------+
|   2000  |                      24 